In [1]:
# Ultra-Precise Wikipedia Family Tree Relationship Extractor
# Focus on visual structure accuracy to eliminate relationship errors

!pip install beautifulsoup4 requests lxml

import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import quote
import pandas as pd

class UltraPreciseFamilyTreeExtractor:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'UltraPreciseFamilyTreeExtractor/1.0 (Educational Purpose)'
        })

    def clean_name(self, text):
        """Ultra-clean name extraction removing all non-name elements"""
        if not text:
            return ""

        # Remove reference numbers and edit links
        text = re.sub(r'\[.*?\]', '', text)

        # Remove ALL date patterns aggressively
        text = re.sub(r'\d{1,2}\s+(January|February|March|April|May|June|July|August|September|October|November|December)', '', text, flags=re.IGNORECASE)
        text = re.sub(r'(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}', '', text, flags=re.IGNORECASE)
        text = re.sub(r'\b\d{4}\b', '', text)
        text = re.sub(r'\(\d{4}[-–]\d{4}\)', '', text)
        text = re.sub(r'\(\d{4}[-–]\)', '', text)
        text = re.sub(r'\([-–]\d{4}\)', '', text)
        text = re.sub(r'\(\d{4}\)', '', text)
        text = re.sub(r'\bb\.\s*\d{4}', '', text)
        text = re.sub(r'\bd\.\s*\d{4}', '', text)
        text = re.sub(r'\br\.\s*\d{4}[-–]\d{4}', '', text)

        # Remove ordinal numbers and regnal numbers more precisely
        text = re.sub(r'\b(I{1,3}|IV|V|VI{1,3}|IX|X|XI{1,3})\b(?!\w)', '', text)

        # Remove titles but keep them as part of disambiguation
        titles_pattern = r'\b(King|Queen|Prince|Princess|Duke|Duchess|Count|Countess|Grand Duke|Grand Duchess|Emperor|Empress|Tsar|Tsarina|Kaiser|Lord|Lady|Sir|Dame|Baron|Baroness|Earl|Marquess)\b'
        text = re.sub(titles_pattern, '', text, flags=re.IGNORECASE)

        # Remove "of [Place]" but be careful with names that include it
        text = re.sub(r'\bof\s+[A-Z][a-zA-Z\s]*(?=\s|$)', '', text)

        # Remove parentheses content
        text = re.sub(r'\([^)]*\)', '', text)

        # Clean punctuation and whitespace
        text = re.sub(r'[,\.\-–;:]+', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()

        # Filter invalid results
        if len(text) < 2 or not re.search(r'[A-Za-z]', text):
            return ""

        # Filter common non-names
        exclude_terms = ['edit', 'show', 'hide', 'more', 'other', 'children', 'issue',
                        'and others', 'see also', 'main article', 'born', 'died', 'aged',
                        'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'with',
                        'see', 'also', 'main', 'article']

        if text.lower().strip() in exclude_terms:
            return ""

        # Only return names that look legitimate
        words = text.split()
        if len(words) == 1 and len(words[0]) < 3:
            return ""

        return text.strip()

    def extract_names_from_cell(self, cell):
        """Extract person names with extreme precision"""
        names = []

        # Priority 1: Wikipedia links to person pages (most reliable)
        links = cell.find_all('a', href=True)
        for link in links:
            href = link.get('href', '')
            # Very strict filtering for person pages
            if ('/wiki/' in href and
                ':' not in href and
                'disambiguation' not in href.lower() and
                'category:' not in href.lower() and
                'file:' not in href.lower() and
                'template:' not in href.lower() and
                'list_of' not in href.lower()):

                # Additional check: link should point to likely person page
                link_text = link.get_text().strip()
                if len(link_text) > 2 and not link_text.isdigit():
                    name = self.clean_name(link_text)
                    if name and len(name) > 2 and name not in names:
                        names.append(name)

        # Only use the most reliable names from links
        if names:
            return names[:2]  # Maximum 2 names per cell to avoid noise

        # Fallback: Look for bold text (but be very careful)
        bold_elements = cell.find_all(['b', 'strong'])
        for bold in bold_elements:
            bold_text = bold.get_text().strip()
            if len(bold_text) > 2 and not bold_text.isdigit():
                name = self.clean_name(bold_text)
                if name and len(name) > 2 and name not in names:
                    names.append(name)

        return names[:1] if names else []  # Even more conservative for fallback

    def is_primary_family_tree_table(self, table):
        """Identify the main family tree table with high precision"""
        table_str = str(table).lower()

        # Strong indicators for family trees
        strong_indicators = ['familytree', 'family tree', 'genealogy']
        class_names = ' '.join(table.get('class', [])).lower()

        for indicator in strong_indicators:
            if indicator in class_names or indicator in table_str:
                return True

        # Check for genealogical connecting characters (family tree visual structure)
        tree_chars = ['├', '└', '┌', '│', '─', '┬', '┼', '┤', '┴']
        tree_char_count = sum(1 for char in tree_chars if char in table_str)

        if tree_char_count >= 3:  # Multiple connecting characters indicate family tree
            return True

        # Check table structure and content
        person_links = table.find_all('a', href=re.compile(r'/wiki/[^:]+'))
        valid_links = []

        for link in person_links:
            href = link.get('href', '')
            text = link.get_text().strip()
            # Very strict validation
            if (len(text) > 2 and
                'category:' not in href.lower() and
                'file:' not in href.lower() and
                'template:' not in href.lower() and
                not text.isdigit()):
                valid_links.append(link)

        rows = table.find_all('tr')

        # Must have significant structure and content
        if len(valid_links) >= 6 and len(rows) >= 3:
            # Additional check: should have some hierarchical structure
            cells_with_people = 0
            for row in rows:
                cells = row.find_all(['td', 'th'])
                for cell in cells:
                    if self.extract_names_from_cell(cell):
                        cells_with_people += 1

            if cells_with_people >= 4:
                return True

        return False

    def create_precise_grid(self, table):
        """Create ultra-precise grid with exact positioning"""
        rows = table.find_all('tr')
        grid = []

        for row_idx, row in enumerate(rows):
            cells = row.find_all(['td', 'th'])
            row_cells = []

            col_position = 0
            for cell in cells:
                names = self.extract_names_from_cell(cell)

                # Only process cells that actually contain people
                if names:
                    colspan = int(cell.get('colspan', 1))
                    rowspan = int(cell.get('rowspan', 1))

                    cell_data = {
                        'names': names,
                        'row': row_idx,
                        'col_start': col_position,
                        'col_end': col_position + colspan - 1,
                        'colspan': colspan,
                        'rowspan': rowspan,
                        'cell_html': str(cell)[:200]  # For debugging structure
                    }

                    row_cells.append(cell_data)

                col_position += int(cell.get('colspan', 1))

            if row_cells:  # Only include rows with actual people
                grid.append(row_cells)

        return grid

    def find_precise_spouse_relationships(self, grid):
        """Find spouse relationships with maximum precision"""
        relationships = []

        for row in grid:
            for cell in row:
                names = cell['names']

                # Case 1: Multiple names in same cell (highest confidence)
                if len(names) == 2:  # Exactly 2 names = most likely spouses
                    relationships.append({
                        'type': 'spouse_of',
                        'person1': names[0],
                        'person2': names[1],
                        'confidence': 'very_high',
                        'source': 'same_cell_two_names'
                    })

                # Case 2: Horizontally adjacent cells (lower confidence, stricter rules)
                elif len(names) == 1:
                    current_name = names[0]
                    current_col_end = cell['col_end']

                    # Look for immediately adjacent cell with exactly one name
                    for other_cell in row:
                        if (other_cell['col_start'] == current_col_end + 1 and
                            len(other_cell['names']) == 1 and
                            other_cell != cell):

                            # Additional validation: both should be in similar position/structure
                            relationships.append({
                                'type': 'spouse_of',
                                'person1': current_name,
                                'person2': other_cell['names'][0],
                                'confidence': 'medium',
                                'source': 'adjacent_cells'
                            })

        return relationships

    def find_precise_parent_child_relationships(self, grid):
        """Find parent-child relationships with strict vertical alignment"""
        relationships = []

        # Only look at immediately consecutive rows (no skipping generations)
        for parent_row_idx in range(len(grid) - 1):
            parent_row = grid[parent_row_idx]
            child_row = grid[parent_row_idx + 1]  # ONLY next row

            for parent_cell in parent_row:
                parent_names = parent_cell['names']
                if not parent_names:
                    continue

                parent_col_start = parent_cell['col_start']
                parent_col_end = parent_cell['col_end']

                # Find children that are vertically aligned
                for child_cell in child_row:
                    child_names = child_cell['names']
                    if not child_names:
                        continue

                    child_col_start = child_cell['col_start']
                    child_col_end = child_cell['col_end']

                    # Calculate overlap - must have significant column overlap
                    overlap_start = max(parent_col_start, child_col_start)
                    overlap_end = min(parent_col_end, child_col_end)
                    overlap = max(0, overlap_end - overlap_start + 1)

                    # Strict alignment requirement
                    parent_width = parent_col_end - parent_col_start + 1
                    child_width = child_col_end - child_col_start + 1
                    min_width = min(parent_width, child_width)

                    # Require at least 50% overlap for relationship
                    if overlap >= min_width * 0.5:
                        for parent_name in parent_names:
                            for child_name in child_names:
                                # Prevent self-relationships
                                if parent_name != child_name:
                                    relationships.append({
                                        'type': 'child_of',
                                        'person1': child_name,
                                        'person2': parent_name,
                                        'confidence': 'high',
                                        'source': f'vertical_alignment_rows_{parent_row_idx}_to_{parent_row_idx+1}'
                                    })

        return relationships

    def validate_relationships(self, relationships):
        """Additional validation to prevent obvious errors"""
        valid_relationships = []

        for rel in relationships:
            person1 = rel['person1'].strip()
            person2 = rel['person2'].strip()

            # Skip self-referential relationships
            if person1 == person2:
                continue

            # Skip if names are too similar (likely same person with slight variations)
            if self.names_too_similar(person1, person2):
                continue

            # Skip if names are too short or look invalid
            if len(person1) < 3 or len(person2) < 3:
                continue

            valid_relationships.append(rel)

        return valid_relationships

    def names_too_similar(self, name1, name2):
        """Check if two names are likely the same person"""
        name1_words = set(name1.lower().split())
        name2_words = set(name2.lower().split())

        # If they share most words, probably same person
        if len(name1_words.intersection(name2_words)) >= min(len(name1_words), len(name2_words)) * 0.8:
            return True

        return False

    def extract_relationships_from_article(self, article_title):
        """Main extraction with ultra-precision"""
        try:
            url = f"https://en.wikipedia.org/wiki/{quote(article_title)}"
            response = self.session.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Find THE main family tree table (not all tables)
            all_tables = soup.find_all('table')
            family_trees = [table for table in all_tables if self.is_primary_family_tree_table(table)]

            print(f"Found {len(family_trees)} primary family tree table(s)")

            all_relationships = []

            for i, table in enumerate(family_trees):
                print(f"Processing table {i + 1}...")

                grid = self.create_precise_grid(table)
                if len(grid) < 2:  # Need at least 2 rows for relationships
                    continue

                print(f"  Grid has {len(grid)} rows with people")

                spouse_rels = self.find_precise_spouse_relationships(grid)
                child_rels = self.find_precise_parent_child_relationships(grid)

                # Validate relationships
                spouse_rels = self.validate_relationships(spouse_rels)
                child_rels = self.validate_relationships(child_rels)

                all_relationships.extend(spouse_rels)
                all_relationships.extend(child_rels)

                print(f"  ✓ {len(spouse_rels)} valid marriage relationships")
                print(f"  ✓ {len(child_rels)} valid parent-child relationships")

            # Final deduplication
            unique_relationships = self.remove_exact_duplicates(all_relationships)

            return unique_relationships

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()
            return []

    def remove_exact_duplicates(self, relationships):
        """Remove duplicate relationships with precision"""
        seen = set()
        unique_rels = []

        for rel in relationships:
            if rel['type'] == 'spouse_of':
                # For spouses, check both directions
                key1 = ('spouse_of', rel['person1'], rel['person2'])
                key2 = ('spouse_of', rel['person2'], rel['person1'])
                if key1 not in seen and key2 not in seen:
                    seen.add(key1)
                    seen.add(key2)
                    unique_rels.append(rel)
            else:
                # For parent-child, maintain direction
                key = (rel['type'], rel['person1'], rel['person2'])
                if key not in seen:
                    seen.add(key)
                    unique_rels.append(rel)

        return unique_rels

    def display_relationships(self, relationships):
        """Display with confidence indicators"""
        if not relationships:
            print("❌ No valid relationships found.")
            return

        spouses = [r for r in relationships if r['type'] == 'spouse_of']
        children = [r for r in relationships if r['type'] == 'child_of']

        print(f"\n{'='*70}")
        print(f"🎯 ULTRA-PRECISE FAMILY RELATIONSHIPS")
        print(f"{'='*70}")

        if spouses:
            print(f"\n💑 MARRIAGES ({len(spouses)}):")
            print("-" * 50)
            for rel in spouses:
                confidence_icon = "🔥" if rel['confidence'] == 'very_high' else "✅" if rel['confidence'] == 'high' else "⚠️"
                print(f"{confidence_icon} {rel['person1']} ↔ {rel['person2']} ({rel['confidence']})")

        if children:
            print(f"\n👨‍👩‍👧‍👦 PARENT-CHILD ({len(children)}):")
            print("-" * 50)
            for rel in children:
                confidence_icon = "🔥" if rel['confidence'] == 'very_high' else "✅" if rel['confidence'] == 'high' else "⚠️"
                print(f"{confidence_icon} {rel['person1']} is child of {rel['person2']} ({rel['confidence']})")

        print(f"\n📊 SUMMARY: {len(spouses)} marriages, {len(children)} parent-child relationships")

# Main execution
def extract_ultra_precise_relationships(article_title="Descendants of Christian IX of Denmark"):
    """Extract relationships with maximum accuracy"""
    extractor = UltraPreciseFamilyTreeExtractor()

    print(f"🔍 Ultra-precise extraction from: {article_title}")
    relationships = extractor.extract_relationships_from_article(article_title)

    extractor.display_relationships(relationships)

    if relationships:
        df = pd.DataFrame(relationships)
        return df[['type', 'person1', 'person2', 'confidence', 'source']]
    else:
        return pd.DataFrame()

# Run the ultra-precise extraction
if __name__ == "__main__":
    df = extract_ultra_precise_relationships("Descendants of Christian IX of Denmark")

    if not df.empty:
        print(f"\n{'='*70}")
        print("📋 DETAILED ANALYSIS:")
        print(f"{'='*70}")
        print(df.to_string(index=False, max_colwidth=30))

# Quick test function
def test_precision(title):
    print(f"\n{'🧪 TESTING: ' + title:=^80}")
    return extract_ultra_precise_relationships(title)

🔍 Ultra-precise extraction from: Descendants of Christian IX of Denmark
Found 14 primary family tree table(s)
Processing table 1...
  Grid has 4 rows with people
  ✓ 1 valid marriage relationships
  ✓ 5 valid parent-child relationships
Processing table 2...
  Grid has 3 rows with people
  ✓ 0 valid marriage relationships
  ✓ 5 valid parent-child relationships
Processing table 3...
  Grid has 6 rows with people
  ✓ 4 valid marriage relationships
  ✓ 23 valid parent-child relationships
Processing table 4...
  Grid has 8 rows with people
  ✓ 5 valid marriage relationships
  ✓ 23 valid parent-child relationships
Processing table 5...
  Grid has 8 rows with people
  ✓ 1 valid marriage relationships
  ✓ 8 valid parent-child relationships
Processing table 6...
  Grid has 7 rows with people
  ✓ 0 valid marriage relationships
  ✓ 8 valid parent-child relationships
Processing table 7...
  Grid has 5 rows with people
  ✓ 3 valid marriage relationships
  ✓ 8 valid parent-child relationships
Proces